# Project Goal:
- Import production and permit data for company and create custom fields requested by CEO 
- Add geographic fields to be used with Tableau

## Read in Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import requests as req
import re

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('precision', 2)

In [3]:
permits = pd.read_csv("Raw Data/DI Magnum NewDrill Permits.CSV")
prod = pd.read_csv("Raw Data/Production Table.CSV")
counties = gp.read_file("zip://Raw Data/UScounties.zip")

## Clean Up Production DataFrame

In [4]:
prod.head()
prod["Producing Status"].value_counts()

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),...,EUR Oil Proxy (Y/N),EUR Oil B-Factor (BE),EUR Gas Months (BE),EUR Gas Proxy (Y/N),EUR Gas B-Factor (BE),EUR Oil Months (Full),EUR Oil B-Factor (Full),EUR Gas Months (Full),EUR Gas B-Factor (Full),Other No.
0,42321323610000,"MAGNUM PRODUCING, LP",ANDERSON,1,MATAGORDA (TX),"10,000",GAS,ACTIVE,V,10700.0,...,True,1.06,599.0,True,1.06,599.0,1.06,599.0,1.06,0
1,42469344440000,"MAGNUM PRODUCING, LP",SPACEK UNIT,1,VICTORIA (TX),MIDDLE WILCOX,GAS,ACTIVE,V,9500.0,...,False,2.00,592.0,False,0.81,592.0,2.00,592.0,0.81,0
2,42245328640000,"MAGNUM PRODUCING, LP",MCLEAN UNIT 4,4,JEFFERSON (TX),HETEROSTEGINA - DISPOSAL,OIL,ACTIVE,V,8560.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,42039333450000,"MAGNUM PRODUCING, LP",DUCROZ,1,BRAZORIA (TX),TOP OF FRIO,OIL,INACTIVE,V,11100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,42321324010000,"MAGNUM PRODUCING, LP",ANDERSON,3,MATAGORDA (TX),HUEBNER,OIL,INACTIVE,D,10600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


INACTIVE                                  501
P & A                                     155
ACTIVE                                     63
WORKED OVER TO ANOTHER ZONE                53
TA                                         14
RECLASSIFIED TO ANOTHER TYPE OF ENTITY     10
ACTIVE INJ                                  8
CANCELLED                                   5
PERMIT                                      3
APPLICATION FOR PERMIT TO DRILL             1
Name: Producing Status, dtype: int64

### Remove P&A Wells and API Trailing 0's

In [5]:
#filtering out wells that have already been P&A by magnum
prod = prod.loc[prod["Producing Status"]!='P & A']

prod["Producing Status"].value_counts()

INACTIVE                                  501
ACTIVE                                     63
WORKED OVER TO ANOTHER ZONE                53
TA                                         14
RECLASSIFIED TO ANOTHER TYPE OF ENTITY     10
ACTIVE INJ                                  8
CANCELLED                                   5
PERMIT                                      3
APPLICATION FOR PERMIT TO DRILL             1
Name: Producing Status, dtype: int64

In [6]:
#testing to see if api number can be cleaned by dividing number by 10,000 to remove trailing 0's
test = prod["API/UWI"].iloc[0]

test

int(test/(10**4))


42321323610000

4232132361

In [7]:
#dividing each api by 10,000 to clean number and remove trailing 0's
prod["API/UWI"] = prod["API/UWI"].apply(lambda x: int(x/(10**4)))

In [8]:
prod.head()

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),...,EUR Oil Proxy (Y/N),EUR Oil B-Factor (BE),EUR Gas Months (BE),EUR Gas Proxy (Y/N),EUR Gas B-Factor (BE),EUR Oil Months (Full),EUR Oil B-Factor (Full),EUR Gas Months (Full),EUR Gas B-Factor (Full),Other No.
0,4232132361,"MAGNUM PRODUCING, LP",ANDERSON,1,MATAGORDA (TX),"10,000",GAS,ACTIVE,V,10700.0,...,True,1.06,599.0,True,1.06,599.0,1.06,599.0,1.06,0
1,4246934444,"MAGNUM PRODUCING, LP",SPACEK UNIT,1,VICTORIA (TX),MIDDLE WILCOX,GAS,ACTIVE,V,9500.0,...,False,2.00,592.0,False,0.81,592.0,2.00,592.0,0.81,0
2,4224532864,"MAGNUM PRODUCING, LP",MCLEAN UNIT 4,4,JEFFERSON (TX),HETEROSTEGINA - DISPOSAL,OIL,ACTIVE,V,8560.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,4203933345,"MAGNUM PRODUCING, LP",DUCROZ,1,BRAZORIA (TX),TOP OF FRIO,OIL,INACTIVE,V,11100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4232132401,"MAGNUM PRODUCING, LP",ANDERSON,3,MATAGORDA (TX),HUEBNER,OIL,INACTIVE,D,10600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### Create Cleaned County Column
* cleaned county column only has name and no state appended at the end

In [9]:
#clean county names | using test row to test logic
prod["County/Parish"].iloc[197][:-5]

prod["Cleaned County"] = prod["County/Parish"].apply(lambda x: x[:-5])

'LIBERTY'

In [10]:
prod.head()

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),...,EUR Oil B-Factor (BE),EUR Gas Months (BE),EUR Gas Proxy (Y/N),EUR Gas B-Factor (BE),EUR Oil Months (Full),EUR Oil B-Factor (Full),EUR Gas Months (Full),EUR Gas B-Factor (Full),Other No.,Cleaned County
0,4232132361,"MAGNUM PRODUCING, LP",ANDERSON,1,MATAGORDA (TX),"10,000",GAS,ACTIVE,V,10700.0,...,1.06,599.0,True,1.06,599.0,1.06,599.0,1.06,0,MATAGORDA
1,4246934444,"MAGNUM PRODUCING, LP",SPACEK UNIT,1,VICTORIA (TX),MIDDLE WILCOX,GAS,ACTIVE,V,9500.0,...,2.00,592.0,False,0.81,592.0,2.00,592.0,0.81,0,VICTORIA
2,4224532864,"MAGNUM PRODUCING, LP",MCLEAN UNIT 4,4,JEFFERSON (TX),HETEROSTEGINA - DISPOSAL,OIL,ACTIVE,V,8560.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,JEFFERSON
3,4203933345,"MAGNUM PRODUCING, LP",DUCROZ,1,BRAZORIA (TX),TOP OF FRIO,OIL,INACTIVE,V,11100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,BRAZORIA
4,4232132401,"MAGNUM PRODUCING, LP",ANDERSON,3,MATAGORDA (TX),HUEBNER,OIL,INACTIVE,D,10600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,MATAGORDA


### Adding Latest BOE/month column

In [11]:
prod["Latest BOE"] = (prod["Latest Gas"]/6) + prod["Latest Oil"]

## Permits Table

In [12]:
permits.head()

,API 10/UWI,District,Filed Date,Approved Date,Expired Date,State/Province,County/Parish,Operator Alias (Legacy),Lease Name,Well Number,...,Bottom Hole Latitude (WGS84),Bottom Hole Longitude (WGS84),Lease Number,Permit Depth UOM,Permit Number,DI Basin,DI Play,DI Subplay,Operator Company Name,Operator Ticker
0,3000564345,SE,NaN,2019-12-13,2021-12-02,NM,CHAVES (NM),MAGNUM PROD,THE PEGGY FEDERAL,113,...,33.54,-104.14,326731,ft,TEMP86977488,NORTHWEST SHELF,NORTHWEST SHELF,NaN,"MAGNUM PRODUCING, LP",NaN
1,4245730898,3,2019-10-03,2019-10-08,2021-10-08,TX,TYLER (TX),MAGNUM PROD,BIG THICKET,1,...,30.56,-94.34,NaN,ft,857731,GULF COAST CENTRAL,NaN,NaN,"MAGNUM PRODUCING, LP",NaN
2,4203933345,3,2019-07-03,2019-07-05,2021-07-05,TX,BRAZORIA (TX),MAGNUM PROD,DUCROZ,1,...,28.90,-95.60,27788,ft,854812,GULF COAST CENTRAL,NaN,NaN,"MAGNUM PRODUCING, LP",NaN
3,4232132426,3,2019-06-25,2019-06-26,2021-06-26,TX,MATAGORDA (TX),MAGNUM PROD,TRULL,1,...,28.77,-96.26,NaN,ft,854558,GULF COAST CENTRAL,NaN,NaN,"MAGNUM PRODUCING, LP",NaN
4,4246934444,2,2019-05-23,2019-05-24,2021-05-24,TX,VICTORIA (TX),MAGNUM PROD,SPACEK UNIT,1,...,29.08,-96.98,290069,ft,853438,GULF COAST WEST,NaN,NaN,"MAGNUM PRODUCING, LP",NaN


In [13]:
#finding which wells in production tables exist in the new drill permits table by magnum
prod["API/UWI"].isin(permits["API 10/UWI"]).head(10)

0     True
1     True
2     True
3     True
4     True
5    False
6     True
7     True
8    False
9    False
Name: API/UWI, dtype: bool

### Creating Column to Track Which Wells were Drilled By Magnum
* formula will check if api number exists in the newdrill permits data table

In [14]:
#creating new column representing whether or not well in production table exists
prod["Magnum Drilled (Y/N)"] = prod["API/UWI"].isin(permits["API 10/UWI"])

In [15]:
#mapping truths and falses to yes or no
prod["Magnum Drilled (Y/N)"] = prod["Magnum Drilled (Y/N)"].map({True: "Y", False: "N"})

In [16]:
#looking at how many wells are marked yes versus length of the entire permits table
prod["Magnum Drilled (Y/N)"].value_counts()


prod["Magnum Drilled (Y/N)"].value_counts()['Y']
len(permits)

print("There are {} permits not found in new drill permits table".format(len(permits)- prod["Magnum Drilled (Y/N)"].value_counts()['Y']))

N    536
Y    122
Name: Magnum Drilled (Y/N), dtype: int64

122

198

There are 76 permits not found in new drill permits table


## Counties Shapefile Cleanup

In [17]:
counties.head()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry
0,Lake of the Woods,Minnesota,27,077,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715..."
1,Ferry,Washington,53,019,53019,"POLYGON ((-118.85163 47.94956, -118.84846 48.4..."
2,Stevens,Washington,53,065,53065,"POLYGON ((-117.43883 48.04412, -117.54219 48.0..."
3,Okanogan,Washington,53,047,53047,"POLYGON ((-118.97209 47.93915, -118.97406 47.9..."
4,Pend Oreille,Washington,53,051,53051,"POLYGON ((-117.43858 48.99992, -117.03205 48.9..."


In [18]:
len(counties)
counties["FIPS"].nunique()

3141

3141

### Reading in States Table Online with Postal Codes to Add to Counties Shapefile

In [19]:
stlink = 'https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696'
stTable = pd.read_html(req.get(stlink).content)[0]

In [20]:
stTable.head()

,Name,Postal Code,FIPS
0,Alabama,AL,1.0
1,Alaska,AK,2.0
2,Arizona,AZ,4.0
3,Arkansas,AR,5.0
4,California,CA,6.0


### Joining Counties Shapefile with State Postal Codes and Cleaning up Duplicate Columns

In [21]:
counties = pd.merge(counties, stTable[["Name", "Postal Code"]], how = "left", left_on="STATE_NAME", right_on="Name")

In [22]:
counties.drop(columns = "Name", inplace=True)

In [23]:
counties.head()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry,Postal Code
0,Lake of the Woods,Minnesota,27,077,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715...",MN
1,Ferry,Washington,53,019,53019,"POLYGON ((-118.85163 47.94956, -118.84846 48.4...",WA
2,Stevens,Washington,53,065,53065,"POLYGON ((-117.43883 48.04412, -117.54219 48.0...",WA
3,Okanogan,Washington,53,047,53047,"POLYGON ((-118.97209 47.93915, -118.97406 47.9...",WA
4,Pend Oreille,Washington,53,051,53051,"POLYGON ((-117.43858 48.99992, -117.03205 48.9...",WA


### Cleaning County Names by Converting to UpperCase for Matching/Joining as a Primary Key

In [24]:
counties["NAME"] = counties["NAME"].apply(lambda x: x.upper())

In [25]:
counties[["Postal Code", "NAME", "FIPS"]]

,Postal Code,NAME,FIPS
0,MN,LAKE OF THE WOODS,27077
1,WA,FERRY,53019
2,WA,STEVENS,53065
3,WA,OKANOGAN,53047
4,WA,PEND OREILLE,53051
5,ID,BOUNDARY,16021
6,MT,LINCOLN,30053
7,MT,FLATHEAD,30029
8,MT,GLACIER,30035
9,MT,TOOLE,30101


In [26]:
prod.head(1)

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),...,EUR Gas Proxy (Y/N),EUR Gas B-Factor (BE),EUR Oil Months (Full),EUR Oil B-Factor (Full),EUR Gas Months (Full),EUR Gas B-Factor (Full),Other No.,Cleaned County,Latest BOE,Magnum Drilled (Y/N)
0,4232132361,"MAGNUM PRODUCING, LP",ANDERSON,1,MATAGORDA (TX),"10,000",GAS,ACTIVE,V,10700.0,...,True,1.06,599.0,1.06,599.0,1.06,0,MATAGORDA,636.0,Y


### Joining Production with Counties Data to Bring Over County FIPS Codes & Cleanup Duplicate Columns

In [27]:
prod = pd.merge(prod, counties[["Postal Code", "NAME", "FIPS"]], how = "left", left_on=["State", "Cleaned County"], right_on = ["Postal Code", "NAME"])

In [28]:
prod.drop(columns = ["Postal Code", "NAME"], inplace = True)

In [29]:
prod.head()

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),...,EUR Gas B-Factor (BE),EUR Oil Months (Full),EUR Oil B-Factor (Full),EUR Gas Months (Full),EUR Gas B-Factor (Full),Other No.,Cleaned County,Latest BOE,Magnum Drilled (Y/N),FIPS
0,4232132361,"MAGNUM PRODUCING, LP",ANDERSON,1,MATAGORDA (TX),"10,000",GAS,ACTIVE,V,10700.0,...,1.06,599.0,1.06,599.0,1.06,0,MATAGORDA,636.0,Y,48321
1,4246934444,"MAGNUM PRODUCING, LP",SPACEK UNIT,1,VICTORIA (TX),MIDDLE WILCOX,GAS,ACTIVE,V,9500.0,...,0.81,592.0,2.00,592.0,0.81,0,VICTORIA,38867.0,Y,48469
2,4224532864,"MAGNUM PRODUCING, LP",MCLEAN UNIT 4,4,JEFFERSON (TX),HETEROSTEGINA - DISPOSAL,OIL,ACTIVE,V,8560.0,...,NaN,NaN,NaN,NaN,NaN,0,JEFFERSON,NaN,Y,48245
3,4203933345,"MAGNUM PRODUCING, LP",DUCROZ,1,BRAZORIA (TX),TOP OF FRIO,OIL,INACTIVE,V,11100.0,...,NaN,NaN,NaN,NaN,NaN,0,BRAZORIA,NaN,Y,48039
4,4232132401,"MAGNUM PRODUCING, LP",ANDERSON,3,MATAGORDA (TX),HUEBNER,OIL,INACTIVE,D,10600.0,...,NaN,NaN,NaN,NaN,NaN,0,MATAGORDA,NaN,Y,48321


In [30]:
prod[prod["FIPS"].isna()]

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),...,EUR Gas B-Factor (BE),EUR Oil Months (Full),EUR Oil B-Factor (Full),EUR Gas Months (Full),EUR Gas B-Factor (Full),Other No.,Cleaned County,Latest BOE,Magnum Drilled (Y/N),FIPS
5,0,"MAGNUM PRODUCING, LP",LA LUW 801958,NaN,(N/A) (LA),NaN,OIL,INACTIVE,U,NaN,...,NaN,NaN,NaN,NaN,NaN,801958,(N/A),NaN,N,NaN
215,0,"MAGNUM PRODUCING, LP",PAINTERS GAS UNIT,NaN,UNKNOWN (TX),3-G-1,GAS,INACTIVE,V,NaN,...,NaN,NaN,NaN,NaN,NaN,0,UNKNOWN,NaN,N,NaN


In [31]:
pd.options.display.max_columns = len(prod.columns)
prod.head()

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),Spud Date,First Prod Date,Last Prod Date,Months Produced,Cum Gas,Daily Gas,Cum Oil,Daily Oil,Latest Oil,Latest Gas,Cum Water,Cum BOE,DI Subplay,First Month Oil,First Month Gas,First 6 Oil,DI Basin,First 6 Gas,First 6 BOE,First 6 Water,DI Play,Prac IP Oil Daily,Prac IP BOE,Prac IP Gas Daily,Prac IP CFGED,Latest Water,Prior 12 Oil,Prior 12 Gas,Last Test Date,Prior 12 Water,Last Flow Pressure,Last WHSIP,2nd Month GOR,Latest GOR,Cum GOR,Last 12 Yield,2nd Month Yield,Latest Yield,Peak Gas,Peak Gas Month No.,Peak Oil,Peak Oil Month No.,Peak BOE,Peak BOE Month No.,Peak MMCFGE,Peak MMCFGE Month No.,Upper Perforation,Lower Perforation,Gas Gravity,Oil Gravity,Completion Date,Well Count,Max Active Wells,Gas Gatherer,Oil Gatherer,Lease Number,Gross Perforated Interval,True Vertical Depth,Field,State,District,AAPG Geologic Province,Section,Country,Township,Range,Surface Latitude (WGS84),Abstract,Surface Longitude (WGS84),Block,OCS Area,PGC Area,Survey,Entity ID,EUR Calc Date,EUR Oil (BE),EUR Oil Delta (BE),EUR Oil Delta Percent (BE),EUR Gas (BE),EUR Gas Delta (BE),EUR Gas Delta Percent (BE),Oil RRR (BE),Gas RRR (BE),EUR Oil (Full),EUR Gas (Full),Oil RRR (Full),Operator (Reported),Gas RRR (Full),Last 12 Oil,Last 12 Gas,Last 12 Water,First Inj Date,Last Inj Date,Last Inj Gas (MCF),Last Inj Water (BBL),Last Inj Other (BBL),Cum Inj Gas (MCF),Cum Inj Water (BBL),Cum Inj Other (BBL),Last Inj Product Types,Inj Product Types,Last Inj Pressure (PSI),EUR Oil Months (BE),EUR Oil Proxy (Y/N),EUR Oil B-Factor (BE),EUR Gas Months (BE),EUR Gas Proxy (Y/N),EUR Gas B-Factor (BE),EUR Oil Months (Full),EUR Oil B-Factor (Full),EUR Gas Months (Full),EUR Gas B-Factor (Full),Other No.,Cleaned County,Latest BOE,Magnum Drilled (Y/N),FIPS
0,4232132361,"MAGNUM PRODUCING, LP",ANDERSON,1,MATAGORDA (TX),"10,000",GAS,ACTIVE,V,10700.0,2013-10-21,2021-04-01,2021-04-01,1.0,3.74e+03,125.0,13.0,0.0,13.0,3738.0,0.0,636.0,NaN,13.0,3738.0,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287538.46,287540.0,3.48,0.0,3.5,3738.0,1.0,13.0,1.0,636.0,1.0,4.0,1.0,9457.0,9971.0,NaN,NaN,2021-04-08,1,1.0,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",290845,514.0,NaN,LUCKY,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,28.97,45,-96.00,NaN,NaN,P-330,"HALL, E",130966595,2021-07-29,528.02,0.0,0.0,1.52e+05,0.0,0.0,515.02,1.48e+05,528.02,1.52e+05,515.02,"MAGNUM PRODUCING, LP",1.48e+05,13.0,3.74e+03,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,599.0,True,1.06,599.0,True,1.06,599.0,1.06,599.0,1.06,0,MATAGORDA,636.0,Y,48321
1,4246934444,"MAGNUM PRODUCING, LP",SPACEK UNIT,1,VICTORIA (TX),MIDDLE WILCOX,GAS,ACTIVE,V,9500.0,2019-06-19,2020-09-01,2021-04-01,8.0,1.57e+06,6504.0,20914.0,86.0,2638.0,217374.0,9347.0,283223.0,NaN,780.0,59527.0,15305.0,GULF COAST WEST,1.11e+06,201111.0,8196.0,NaN,65.0,841.0,4657.0,5.05e+06,545.0,NaN,NaN,2020-11-19,NaN,3451.0,4600.0,71570.0,75253.66,75250.0,13.29,14.0,12.1,241644.0,7.0,3760.0,3.0,43245.0,7.0,259.0,7.0,9270.0,9312.0,0.66,46.0,2020-02-23,1,1.0,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",290069,42.0,NaN,HELEN GOHLKE,TX,02,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,29.08,175,-96.98,NaN,NaN,P-330,"HINES, D",130836314,2021-07-29,285288.25,0.0,0.0,7.96e+06,0.0,0.0,264374.25,6.38e+06,285288.25,7.96e+06,264374.25,"MAGNUM PRODUCING, LP",6.38e+06,20914.0,1.57e+06,9347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,592.0,False,2.00,592.0,False,0.81,592.0,2.00,592.0,0.81,0,VICTORIA,38867.0,Y,48469
2,4224532864,"MAGNUM PRODUCING, LP",MCLEAN UNIT 4,4,JEFFERSON (TX),HETEROSTEGINA - DISPOSAL,OIL,ACTIVE,V,8560.0,2018-05-08,2019-07-01,2021-04-01,21.0,2.22e+04,NaN,32557.0,55.0,2005.0,NaN,0.0,36263.0,NaN,938.0,10397.0,6285.0,GULF COAST CENTRAL,2.03e+04,9671.0,0.0,NaN,4.0,40.0,220.0,2.42e+05,0.0,14221.0,22239.0,2020-10-27,0.0,NaN,NaN,58700.0,NaN,680.0,NaN,17.0,NaN,10397.0,1

In [32]:
#finding rows with no lease name
prod[prod["Well/Lease Name"].isna()]

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),Spud Date,First Prod Date,Last Prod Date,Months Produced,Cum Gas,Daily Gas,Cum Oil,Daily Oil,Latest Oil,Latest Gas,Cum Water,Cum BOE,DI Subplay,First Month Oil,First Month Gas,First 6 Oil,DI Basin,First 6 Gas,First 6 BOE,First 6 Water,DI Play,Prac IP Oil Daily,Prac IP BOE,Prac IP Gas Daily,Prac IP CFGED,Latest Water,Prior 12 Oil,Prior 12 Gas,Last Test Date,Prior 12 Water,Last Flow Pressure,Last WHSIP,2nd Month GOR,Latest GOR,Cum GOR,Last 12 Yield,2nd Month Yield,Latest Yield,Peak Gas,Peak Gas Month No.,Peak Oil,Peak Oil Month No.,Peak BOE,Peak BOE Month No.,Peak MMCFGE,Peak MMCFGE Month No.,Upper Perforation,Lower Perforation,Gas Gravity,Oil Gravity,Completion Date,Well Count,Max Active Wells,Gas Gatherer,Oil Gatherer,Lease Number,Gross Perforated Interval,True Vertical Depth,Field,State,District,AAPG Geologic Province,Section,Country,Township,Range,Surface Latitude (WGS84),Abstract,Surface Longitude (WGS84),Block,OCS Area,PGC Area,Survey,Entity ID,EUR Calc Date,EUR Oil (BE),EUR Oil Delta (BE),EUR Oil Delta Percent (BE),EUR Gas (BE),EUR Gas Delta (BE),EUR Gas Delta Percent (BE),Oil RRR (BE),Gas RRR (BE),EUR Oil (Full),EUR Gas (Full),Oil RRR (Full),Operator (Reported),Gas RRR (Full),Last 12 Oil,Last 12 Gas,Last 12 Water,First Inj Date,Last Inj Date,Last Inj Gas (MCF),Last Inj Water (BBL),Last Inj Other (BBL),Cum Inj Gas (MCF),Cum Inj Water (BBL),Cum Inj Other (BBL),Last Inj Product Types,Inj Product Types,Last Inj Pressure (PSI),EUR Oil Months (BE),EUR Oil Proxy (Y/N),EUR Oil B-Factor (BE),EUR Gas Months (BE),EUR Gas Proxy (Y/N),EUR Gas B-Factor (BE),EUR Oil Months (Full),EUR Oil B-Factor (Full),EUR Gas Months (Full),EUR Gas B-Factor (Full),Other No.,Cleaned County,Latest BOE,Magnum Drilled (Y/N),FIPS
57,4224130695,"MAGNUM PRODUCING, LP",NaN,1W,JASPER (TX),YEGUA 7300,SWD,CANCELLED,D,11357.0,2003-03-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7135.0,7833.0,NaN,NaN,NaN,1,NaN,NaN,NaN,000111278,698.0,NaN,BUNA,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,65,USA,NaN,NaN,30.43,510,-94.02,NaN,NaN,P-330,T&NO RR CO,128925762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MAGNUM PRODUCING, LP",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,0,JASPER,NaN,N,48241
74,4248101745,"MAGNUM PRODUCING, LP",NaN,3,WHARTON (TX),MIOCENE 4300,SWD,PERMIT,V,7777.0,1936-03-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3700.0,4800.0,NaN,NaN,1968-04-01,1,NaN,NaN,NaN,000105153,1100.0,NaN,LOUISE,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,23,USA,NaN,NaN,29.06,287,-96.39,NaN,NaN,P-330,M&C,128275019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MAGNUM PRODUCING, LP",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,0,WHARTON,NaN,N,48481
75,4240932532,"MAGNUM PRODUCING, LP",NaN,1,SAN PATRICIO (TX),4000,SWD,PERMIT,V,7423.0,1998-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST WEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2980.0,5820.0,NaN,NaN,1998-02-05,1,NaN,NaN,NaN,000104133,2840.0,NaN,TAFT,TX,04,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,28.00,129,-97.42,7,NaN,P-330,"GARCIA, J",128274915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MAGNUM PRODUCING, LP",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,0,SAN PATRICIO,NaN,N,48409
76,4240901423,"MAGNUM PRODUCING, LP",NaN,2D,SAN PATRICIO (TX),3900,SWD,PERMIT,V,4015.0,1951-01-01,NaN,NaN,NaN,NaN,N

In [33]:
#dropping rows in table with no well name since they were all SWD wells that were permits or cancelled
prod.drop(index = prod[prod["Well/Lease Name"].isna()].index, inplace = True)

## Exporting Prepared Production Table to File

In [34]:
prod.to_excel("Data for Tableau - Do not Edit Files in Here/Prepared Production Table.xlsx", index = False)

In [35]:
break

SyntaxError: 'break' outside loop (<ipython-input-35-6aaf1f276005>, line 4)

<hr>

# Stopping Point - Code below is Irrelevant as of 6/25/21

<hr>

## Merging Well List with Sell Information

In [ ]:
import os

In [ ]:
sellInfoDF = pd.read_excel('VOID Well - Sell Intention - Table.xlsx')
sellInfoDF.drop(index = sellInfoDF[sellInfoDF["Well/Lease Name"].isna()].index, inplace = True)

In [ ]:
#source code to add sheet to existing workbook is found here: 
#https://stackoverflow.com/questions/42370977/how-to-save-a-new-sheet-in-an-existing-excel-file-using-pandas

from openpyxl import load_workbook

#path of where excel file is to add sheet to
path = os.path.abspath('Data for Tableau - Do not Edit Files in Here/Prepared Production Table.xlsx')

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book


sellInfoDF.to_excel(writer, sheet_name = 'Well Sale List', index = False)
writer.save()
writer.close()


# Merging Coworker's (Anish's) Version


In [ ]:
#Storing sheet names needed to unify tables
sheets = pd.ExcelFile("Magnum TX RRC Sell Intention.xlsx").sheet_names[:3]
sheets

In [ ]:
dfList = []


for sheet in sheets:
    temp = pd.read_excel('Magnum TX RRC Sell Intention.xlsx', header = 1, sheet_name=sheet)
    dfList.append(temp)
    
anishDF = pd.concat(dfList, sort=False)    

In [ ]:
sellInfoDF.head(2)
sellInfoDF.info()
anishDF.head(2)
anishDF.info()

In [ ]:
leftTable = sellInfoDF[["API/UWI", "Well/Lease Name", "Sale Intention"]]
rightTable = anishDF.drop(columns=["Sale Intention", "Lease Name", "On Schedule"])
rightTable.rename(columns = {"API": "API/UWI"}, inplace = True)

In [ ]:
mergedDF = pd.merge(leftTable, rightTable, how = "left", on = "API/UWI")
mergedDF